In [1]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from itertools import chain
import time
import requests
import random
import pandas as pd
import matplotlib.pyplot as plt
from goose3 import Goose

In [2]:
program_languages=['bash','r','python','java','c++','c#', 'f#', 'ruby','perl','matlab','javascript','scala','php']
analysis_software=['alteryx', 'excel','tableau', 'power bi', 'powerbi', 'qlikview', 'qlik','d3.js','sas','sap','spss','d3','saas','pandas','numpy','scipy','sps','spotfire','scikits.learn','splunk','powerpoint','h2o']
bigdata_tool=['hadoop','mapreduce','spark','pig','hive','shark','oozie','zookeeper','flume','mahout']
databases=['sql','nosql','hbase','cassandra','mongodb','mysql','mssql','postgresql','oracle db','rdbms']
machine_learning = ['keras', 'scikit-learn', 'tensorflow', 'caffe', 'datarobot', 'theano', 'torch']
overall_dict = program_languages + analysis_software + bigdata_tool + databases + machine_learning

communicative_skills = ["analytisch vermogen", "projectmanagement", "proactief", "communicatieve vaardigheden", "uitdrukkingsvaardigheden", "agile", "scrum", 
"advies", "adviseur", "werkt samen", "werkt nauw samen", "samenwerken", "analytisch", "planmatig", "zelfstandig", "enthousiast", "vertalen", "inventariseren", "analyseren", 
"realiseren", "ondernemen", "adviseer", "oplossingsgericht", "klantgericht", "gedreven", "creatief", "meedenkend", "resultaatgericht", "oplossingsgericht",
"prioriteren", "presenteren"]

In [3]:
def keywords_extract(url):
    g = Goose()
    article = g.extract(url=url)
    text = article.cleaned_text
    text = re.sub("[^a-zA-Z+3]"," ", text) #get rid of things that aren't words; 3 for d3 and + for c++
    text = text.lower().split()
    stops = set(stopwords.words("english")) #filter out stop words in english language
    text = [w for w in text if not w in stops]
    text = list(set(text))
    keywords = [str(word) for word in text if word in overall_dict]
    return keywords

In [4]:
def keywords_f(soup_obj):
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    text = soup_obj.get_text() 
    lines = (line.strip() for line in text.splitlines()) # break into line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    try:
        text = text.decode('utf-8') # Need this as some websites aren't formatted
    except:                                                          
        return                                                       
    text = re.sub("[^a-zA-Z+3]"," ", text)  
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    text = text.lower()
    text = re.sub(('power bi'), 'powerbi', text) # catch power bi before tokenization
    text = text.split()  
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    text = list(set(text)) #only care about if a word appears, don't care about the frequency
    keywords = [str(word) for word in text if word in overall_dict] #if a skill keyword is found, return it.
    return keywords

In [5]:
def communication_skills_f(soup_obj):
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    text = soup_obj.get_text() 
    lines = (line.strip() for line in text.splitlines()) # break into line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    try:
        text = text.decode('utf-8') # Need this as some websites aren't formatted
    except:                                                          
        return                                                       
    text = re.sub("[^a-zA-Z+3]"," ", text)  
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    text = text.lower()
    skills = [word for word in communicative_skills if word in text] 
    return skills

In [6]:
base_url = "http://www.indeed.nl"    
#change the start_url can scrape different cities.
start_url = "http://www.indeed.nl/jobs?q=BI+consultant&l="
resp = requests.get(start_url)
start_soup = BeautifulSoup(resp.content, 'lxml', from_encoding='utf-8')
urls=start_soup.find_all('a',{'rel':'nofollow','target':'_blank'})
urls = [link['href'] for link in urls] 
num_found = start_soup.find(id = 'searchCount').string.encode('utf-8').split() #this returns the total number of results
num_jobs = num_found[-1].decode('utf-8').split(',')
num_jobs = [x.replace('.', '') for x in num_jobs]

if len(num_jobs)>=2:
    num_jobs = int(num_jobs[0]) * 1000 + int(num_jobs[1])
else:
    num_jobs = int(num_jobs[0])
num_pages = int(num_jobs/10) #calculates how many pages needed to do the scraping

job_keywords=[]
print('There are %d jobs found and we need to extract %d pages.'%(num_jobs,num_pages))
print('extracting first page of job searching results')

There are 807 jobs found and we need to extract 80 pages.
extracting first page of job searching results


In [7]:
# # prevent the driver stopping due to the unexpectedAlertBehaviour.
# webdriver.DesiredCapabilities.FIREFOX["unexpectedAlertBehaviour"] = "accept"
# get_info = True
# driver = webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
# # set a page load time limit so that don't have to wait forever if the links are broken.
# driver.set_page_load_timeout(15)
# for i in range(len(urls)):
#     get_info = True
#     try:
#         driver.get(base_url+urls[i])
#     except TimeoutException:
#         get_info = False
#         continue
#     j = random.randint(1000,2200)/1000.0
#     time.sleep(j) #waits for a random time so that the website don't consider you as a bot
#     if get_info:
#         soup=BeautifulSoup(driver.page_source, 'lxml')
#         print('extracting %d job keywords...' % i)
#         soup_string = str(soup)
#         single_job = keywords_f(soup)
#         other_job = communication_skills_f(soup)
#         print(single_job, other_job, len(soup))
#         print(driver.current_url)
#         job_keywords.append([driver.current_url,single_job, other_job])
# driver.quit()

# skills_dict = [w[1] and w[2] for w in job_keywords]
# skills_dict = list(chain.from_iterable(skills_dict))

In [8]:
# dict={}
# for word in skills_dict:
#     if not word in dict:
#         dict[word]=1
#     else:
#         dict[word]+=1

# Result = pd.DataFrame()
# Result['Skill'] = dict.keys()
# Result['Count'] = dict.values()
# print(Result)

In [ ]:
# prevent the driver stopping due to the unexpectedAlertBehaviour.
webdriver.DesiredCapabilities.FIREFOX["unexpectedAlertBehaviour"] = "accept"
get_info = True
driver = webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
# set a page load time limit so that don't have to wait forever if the links are broken.
driver.set_page_load_timeout(15)

for k in range(1,num_pages + 1):
#this 5 pages reopen the browser is to prevent connection refused error.
    if k%5==0.0:
        driver.quit()
        driver=webdriver.Firefox(executable_path=r'C:\Users\emily\Documents\ML course\geckodriver.exe')
        driver.set_page_load_timeout(15)

    current_url = start_url + "&start=" + str(k*10)
    print('extracting %d page of job searching results...' % k)
    resp = requests.get(current_url)
    current_soup = BeautifulSoup(resp.content, 'lxml')
    current_urls = current_soup.findAll('a',{'rel':'nofollow','target':'_blank'})
    current_urls = [link['href'] for link in current_urls]
    
    for i in range(len(current_urls)):
        get_info = True
        try:
            driver.get(base_url + current_urls[i])
        except TimeoutException:
            get_info = False
            continue 
        j = random.randint(1500,3200)/1000.0
        time.sleep(j) #waits for a random time
        if get_info:
            soup=BeautifulSoup(driver.page_source, 'lxml')
            print('extracting %d job keywords...' % i)
        single_job = keywords_f(soup)
        other_job = communication_skills_f(soup)
        print(single_job, other_job, len(soup))
        print(driver.current_url)
        job_keywords.append([driver.current_url,single_job, other_job])

            
driver.quit()

skills_dict = [w[1] and w[2] for w in job_keywords]
skills_dict = list(chain.from_iterable(skills_dict))
dict={}
for word in skills_dict:
    if not word in dict:
        dict[word]=1
    else:
        dict[word]+=1

Result = pd.DataFrame()
Result['Skill'] = dict.keys()
Result['Count'] = dict.values()
print(Result)

extracting 1 page of job searching results...
extracting 0 job keywords...
['sap'] ['agile', 'vertalen', 'inventariseren', 'analyseren', 'realiseren', 'ondernemen', 'gedreven', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=638eb3aa8de6ce6f&tk=1ckuo3k7m950ib3k&from=serp&alid=3&advn=6124692058421937
extracting 1 job keywords...
[] ['analytisch', 'zelfstandig', 'enthousiast', 'vertalen', 'analyseren', 'klantgericht', 'creatief'] 1
https://www.indeed.nl/vacature-bekijken?jk=209af06ed43c644b&tk=1ckuo3nbr950ie42&from=serp&alid=3&advn=2421485629829485
extracting 2 job keywords...
[] [] 1
https://www.indeed.nl/vacature-bekijken?jk=7593cc50f485ec4f&tk=1ckuo3q41950i8n7&from=serp&alid=3&advn=2421485629829485
extracting 3 job keywords...
['excel', 'tableau', 'sql'] ['agile', 'advies', 'samenwerken', 'ondernemen', 'adviseer', 'gedreven'] 1
https://www.indeed.nl/vacature-bekijken?jk=1006db74a19f404a&from=serp&vjs=3
extracting 4 job keywords...
['hadoop'] ['analytisch vermogen', 'c

extracting 9 job keywords...
['qlikview', 'qlik', 'sql', 'sap'] ['vertalen', 'analyseren'] 1
https://www.indeed.nl/vacature-bekijken?jk=1b48cabc6d81719d&from=serp&vjs=3
extracting 10 job keywords...
['hadoop'] ['proactief', 'samenwerken', 'vertalen', 'realiseren', 'adviseer'] 1
https://www.indeed.nl/vacature-bekijken?jk=d79311ab14a80c50&from=serp&vjs=3
extracting 11 job keywords...
['qlik'] ['werkt samen', 'analyseren', 'realiseren'] 1
https://www.indeed.nl/vacature-bekijken?jk=a084b2aebeab659d&from=serp&vjs=3
extracting 12 job keywords...
['powerbi', 'tableau', 'sql'] ['analytisch vermogen', 'scrum', 'analytisch', 'realiseren'] 1
https://www.indeed.nl/vacature-bekijken?jk=12fa3f497cac58c4&from=serp&vjs=3
extracting 13 job keywords...
['sap'] ['agile', 'vertalen', 'inventariseren', 'analyseren', 'realiseren', 'ondernemen', 'gedreven', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=638eb3aa8de6ce6f&tk=1ckuo8csd950idmu&from=serp&alid=3&advn=6124692058421937
extracting 1

extracting 0 job keywords...
['sap'] ['agile', 'vertalen', 'inventariseren', 'analyseren', 'realiseren', 'ondernemen', 'gedreven', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=638eb3aa8de6ce6f&tk=1ckuoccku9n0vero&from=serp&alid=3&advn=6124692058421937
extracting 1 job keywords...
[] [] 1
https://www.indeed.nl/vacature-bekijken?jk=7593cc50f485ec4f&tk=1ckuocg2b9n0vaao&from=serp&alid=3&advn=2421485629829485
extracting 2 job keywords...
['hadoop', 'python', 'r', 'java', 'nosql', 'sql', 'c++', 'spark'] ['scrum', 'advies', 'adviseur', 'werkt samen', 'gedreven'] 1
https://www.indeed.nl/vacature-bekijken?jk=24115a299a2e1531&tk=1ckuocjbp9n0veq5&from=serp&alid=3&advn=8245509577778884
extracting 3 job keywords...
['sap'] ['proactief', 'agile', 'scrum'] 1
https://www.indeed.nl/vacature-bekijken?jk=a97867cf3ce8314b&from=serp&vjs=3
extracting 4 job keywords...
[] ['agile', 'scrum', 'analytisch', 'zelfstandig', 'vertalen', 'realiseren', 'prioriteren'] 1
https://www.indeed.nl/vacat

extracting 8 job keywords...
[] [] 1
https://www.indeed.nl/vacature-bekijken?jk=d927c92a2d7674a7&from=serp&vjs=3
extracting 9 job keywords...
['powerbi', 'tableau', 'qlik', 'sql', 'sap', 'r'] ['proactief', 'advies', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=37a3bbc1b237613a&from=serp&vjs=3
extracting 10 job keywords...
['excel'] ['projectmanagement', 'adviseer'] 1
https://www.indeed.nl/vacature-bekijken?jk=48f90d2de7b1c71d&from=serp&vjs=3
extracting 11 job keywords...
['powerbi', 'qlikview', 'sap'] [] 1
https://www.indeed.nl/vacature-bekijken?jk=3b9a4787e5cfba58&from=serp&vjs=3
extracting 12 job keywords...
['powerbi', 'qlik', 'sql', 'sap'] ['communicatieve vaardigheden', 'analytisch', 'vertalen', 'oplossingsgericht', 'klantgericht', 'gedreven', 'oplossingsgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=3820567ae64fd9a7&from=serp&vjs=3
extracting 13 job keywords...
['tableau', 'qlik', 'hadoop', 'sas', 'r'] ['projectmanagement', 'samenwerken', 'analytisch',

extracting 15 job keywords...
['powerbi', 'sql'] ['advies', 'werkt samen', 'zelfstandig', 'enthousiast'] 1
https://www.indeed.nl/vacature-bekijken?jk=0a8fa72b3f5e0492&tk=1ckuokikd9n0v9i4&from=serp&alid=3&advn=2180790811884654
extracting 11 page of job searching results...
extracting 0 job keywords...
[] ['analytisch', 'zelfstandig', 'enthousiast', 'vertalen', 'analyseren', 'klantgericht', 'creatief'] 1
https://www.indeed.nl/vacature-bekijken?jk=209af06ed43c644b&tk=1ckuokn019n0valh&from=serp&alid=3&advn=2421485629829485
extracting 1 job keywords...
['sql', 'r'] [] 1
https://www.indeed.nl/vacature-bekijken?jk=9a8fdc01f707b1fe&tk=1ckuokr079n0vd1t&from=serp&alid=3&advn=713903982049763
extracting 2 job keywords...
[] [] 1
https://www.indeed.nl/vacature-bekijken?jk=7593cc50f485ec4f&tk=1ckuoktc99n0vbpt&from=serp&alid=3&advn=2421485629829485
extracting 3 job keywords...
['powerbi', 'sql', 'r'] ['werkt samen', 'enthousiast', 'vertalen', 'gedreven'] 1
https://www.indeed.nl/vacature-bekijken?jk=6

...

extracting 3 job keywords...
['powerbi'] ['analytisch', 'zelfstandig', 'analyseren'] 1
https://www.indeed.nl/vacature-bekijken?jk=c8413feb9b726124&from=serp&vjs=3
extracting 4 job keywords...
['powerbi', 'java'] ['communicatieve vaardigheden', 'agile', 'scrum', 'analytisch', 'zelfstandig', 'analyseren', 'adviseer'] 1
https://www.indeed.nl/vacature-bekijken?jk=9a80bcded3c3d22e&from=serp&vjs=3
extracting 5 job keywords...
['sap'] ['analyseren', 'gedreven'] 1
https://www.indeed.nl/vacature-bekijken?jk=d9fd93e3d6ea6089&from=serp&vjs=3
extracting 6 job keywords...
['sap', 'sql'] ['proactief', 'agile', 'scrum', 'werkt samen', 'analytisch', 'planmatig', 'enthousiast', 'gedreven', 'creatief'] 1
https://www.indeed.nl/vacature-bekijken?jk=43193b62f0c97167&from=serp&vjs=3
extracting 7 job keywords...
[] ['analytisch vermogen', 'advies', 'werkt samen', 'analytisch', 'zelfstandig', 'adviseer'] 1
https://www.indeed.nl/vacature-bekijken?jk=55dea2660963b280&from=serp&vjs=3
extracting 8 job keywords...

extracting 10 job keywords...
['java', 'sql', 'r'] ['proactief', 'agile', 'scrum', 'advies', 'werkt samen', 'zelfstandig', 'adviseer', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=c9617e85bd77f450&from=serp&vjs=3
extracting 11 job keywords...
[] ['agile', 'analytisch', 'zelfstandig', 'ondernemen', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=71b0aff1e94a23b0&from=serp&vjs=3
extracting 12 job keywords...
['sap'] [] 1
https://www.indeed.nl/vacature-bekijken?jk=6371d78b1ab29691&from=serp&vjs=3
extracting 13 job keywords...
[] ['werkt samen'] 1
https://www.indeed.nl/vacature-bekijken?jk=c47b88b7bf473cd6&from=serp&vjs=3
extracting 14 job keywords...
['sap'] ['agile', 'vertalen', 'inventariseren', 'analyseren', 'realiseren', 'ondernemen', 'gedreven', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=638eb3aa8de6ce6f&tk=1ckuqia7v92kv8bd&from=serp&alid=3&advn=6124692058421937
extracting 15 job keywords...
[] [] 1
https://www.indeed.nl/vaca

extracting 1 job keywords...
['powerbi', 'sql', 'r'] ['werkt samen', 'enthousiast', 'vertalen', 'gedreven'] 1
https://www.indeed.nl/vacature-bekijken?jk=6d028a6a0c87e41c&q=BI+consultant&from=web&advn=2180790811884654&sjdu=XGcMNTSedHgn69ge8Rv0-QczTG3nlC0EU2f2klO71b63VlKMcsXkzpKWOnuUTRA28Y-JNPk6g2f94QTYfUqzrRFHKK5gkv3k4Jv2v0R8T9NZ3fASCkljWXRWvynqtS9y2vQaw46jDDkaMfs5GwNu36PqQA1LV3jhFCCDzCMd938&acatk=1ckuqm67c92kva31&pub=4a1b367933fd867b19b072952f68dceb&vjs=3
extracting 2 job keywords...
['powerbi', 'sql'] ['advies', 'werkt samen', 'zelfstandig', 'enthousiast'] 1
https://www.indeed.nl/vacature-bekijken?jk=0a8fa72b3f5e0492&tk=1ckuqm94g92kvc3f&from=serp&alid=3&advn=2180790811884654
extracting 3 job keywords...
[] ['analytisch', 'zelfstandig', 'enthousiast', 'vertalen', 'analyseren', 'klantgericht', 'creatief'] 1
https://www.indeed.nl/vacature-bekijken?jk=209af06ed43c644b&tk=1ckuqmbnv92kvaam&from=serp&alid=3&advn=2421485629829485
extracting 4 job keywords...
['excel', 'sap'] ['analytisch', 'z

extracting 6 job keywords...
[] ['analytisch', 'klantgericht', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=f05c77a72ec2db76&from=serp&vjs=3
extracting 7 job keywords...
[] ['samenwerken', 'zelfstandig', 'enthousiast'] 1
https://www.indeed.nl/vacature-bekijken?jk=f5abc259dd2cbad5&from=serp&vjs=3
extracting 8 job keywords...
['qlikview', 'sap'] [] 1
https://www.indeed.nl/vacature-bekijken?jk=2c8fab3c29707058&from=serp&vjs=3
extracting 9 job keywords...
['qlikview', 'sql', 'sap'] ['samenwerken', 'realiseren'] 1
https://www.indeed.nl/vacature-bekijken?jk=cb2356627542037d&from=serp&vjs=3
extracting 10 job keywords...
['powerbi', 'qlik', 'sas'] ['zelfstandig', 'enthousiast', 'oplossingsgericht', 'gedreven', 'oplossingsgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=548128676a98871b&from=serp&vjs=3
extracting 11 job keywords...
['sap'] ['presenteren'] 1
https://www.indeed.nl/vacature-bekijken?jk=a3356bce3bde29b2&from=serp&vjs=3
extracting 12 job keywords...
['java'

extracting 15 job keywords...
['sap'] ['agile', 'vertalen', 'inventariseren', 'analyseren', 'realiseren', 'ondernemen', 'gedreven', 'resultaatgericht'] 1
https://www.indeed.nl/vacature-bekijken?jk=638eb3aa8de6ce6f&tk=1ckuqulk19nsj9qt&from=serp&alid=3&advn=6124692058421937
extracting 54 page of job searching results...
extracting 0 job keywords...
['powerbi', 'sql'] ['advies', 'werkt samen', 'zelfstandig', 'enthousiast'] 1
https://www.indeed.nl/vacature-bekijken?jk=0a8fa72b3f5e0492&tk=1ckuquovd9nsja2v&from=serp&alid=3&advn=2180790811884654
extracting 1 job keywords...
[] ['analytisch', 'zelfstandig', 'enthousiast', 'vertalen', 'analyseren', 'klantgericht', 'creatief'] 1
https://www.indeed.nl/vacature-bekijken?jk=209af06ed43c644b&tk=1ckuqurn79nsj9j6&from=serp&alid=3&advn=2421485629829485
extracting 2 job keywords...
[] [] 1
https://www.indeed.nl/vacature-bekijken?jk=7593cc50f485ec4f&tk=1ckuquve99nsjeub&from=serp&alid=3&advn=2421485629829485
extracting 3 job keywords...
['hadoop', 'python

In [ ]:
Result.to_csv("skills_for_BIconsultant.csv")